In [1]:
import numpy as np
import re
from IPython.display import clear_output

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

/Users/davidfoster/.virtualenvs/gdl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
load_saved_model = False
train_model = True

In [3]:
token_type = 'word'

In [4]:
#load in the text and perform some cleanup

seq_length = 20

filename = "./data/text/aesop.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()
    
start_story = '| ' * seq_length
    
text = start_story + text
text = text.lower()
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)





In [5]:
len(text)

213676

In [6]:
text

' | | | | | | | | | | | | | | | | | | | | the fox and the grapes . a hungry fox saw some fine bunches of grapes hanging from a vine that was trained along a high trellis , and did his best to reach them by jumping as high as he could into the air . but it was all in vain , for they were just out of reach : so he gave up trying , and walked away with an air of dignity and unconcern , remarking , " i thought those grapes were ripe , but i see now they are quite sour . " | | | | | | | | | | | | | | | | | | | | the goose that laid the golden eggs . a man and his wife had the good fortune to possess a goose which laid a golden egg every day . lucky though they were , they soon began to think they were not getting rich fast enough , and , imagining the bird must be made of gold inside , they decided to kill it in order to secure the whole store of precious metal at once . but when they cut it open they found it was just like any other goose . thus , they neither got rich all at once , as the

In [7]:

if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)
    
    
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]


In [8]:
total_words

4169

In [9]:
tokenizer.word_index

{'|': 1,
 ',': 2,
 'the': 3,
 'and': 4,
 '.': 5,
 'a': 6,
 'to': 7,
 '"': 8,
 'of': 9,
 'he': 10,
 'his': 11,
 'was': 12,
 'you': 13,
 'in': 14,
 'him': 15,
 'it': 16,
 'for': 17,
 'i': 18,
 'but': 19,
 'that': 20,
 'with': 21,
 'said': 22,
 'they': 23,
 'at': 24,
 'as': 25,
 'when': 26,
 'on': 27,
 'so': 28,
 'by': 29,
 ':': 30,
 'one': 31,
 'had': 32,
 ';': 33,
 'be': 34,
 'them': 35,
 'up': 36,
 'all': 37,
 'who': 38,
 '-': 39,
 'her': 40,
 'my': 41,
 'is': 42,
 'out': 43,
 'me': 44,
 'not': 45,
 'your': 46,
 'were': 47,
 'which': 48,
 'have': 49,
 'their': 50,
 'no': 51,
 'lion': 52,
 'she': 53,
 'if': 54,
 'into': 55,
 'fox': 56,
 'an': 57,
 'what': 58,
 'ass': 59,
 '!': 60,
 'time': 61,
 'from': 62,
 'are': 63,
 'came': 64,
 '?': 65,
 'man': 66,
 'do': 67,
 'about': 68,
 'there': 69,
 'wolf': 70,
 'then': 71,
 'upon': 72,
 'himself': 73,
 'this': 74,
 'day': 75,
 'would': 76,
 'some': 77,
 'once': 78,
 'will': 79,
 'very': 80,
 'went': 81,
 'well': 82,
 'than': 83,
 'down': 84,
 

In [10]:
token_list

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 56,
 4,
 3,
 940,
 5,
 6,
 382,
 56,
 94,
 77,
 216,
 1557,
 9,
 940,
 941,
 62,
 6,
 581,
 20,
 12,
 2226,
 162,
 6,
 359,
 2227,
 2,
 4,
 158,
 11,
 250,
 7,
 383,
 35,
 29,
 1176,
 25,
 359,
 25,
 10,
 88,
 55,
 3,
 582,
 5,
 19,
 16,
 12,
 37,
 14,
 785,
 2,
 17,
 23,
 47,
 96,
 43,
 9,
 383,
 30,
 28,
 10,
 170,
 36,
 425,
 2,
 4,
 426,
 89,
 21,
 57,
 582,
 9,
 1558,
 4,
 2228,
 2,
 1559,
 2,
 8,
 18,
 144,
 260,
 940,
 47,
 1177,
 2,
 19,
 18,
 90,
 115,
 23,
 63,
 360,
 2229,
 5,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1178,
 20,
 519,
 3,
 659,
 660,
 5,
 6,
 66,
 4,
 11,
 520,
 32,
 3,
 91,
 384,
 7,
 2230,
 6,
 1178,
 48,
 519,
 6,
 659,
 1179,
 159,
 75,
 5,
 942,
 385,
 23,
 47,
 2,
 23,
 171,
 126,
 7,
 204,
 23,
 47,
 45,
 386,
 474,
 361,
 177,
 2,
 4,
 2,
 2231,
 3,
 229,
 186,
 34,
 109,
 9,
 786,
 427,
 2,
 23,
 943,
 7,
 237,
 16,
 14

In [11]:
def generate_sequences(token_list, step):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    

    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)


Number of sequences: 50396 



In [12]:
X.shape

(50396, 20)

In [13]:
y.shape

(50396, 4169)

## Define the LSTM model

In [14]:
if load_saved_model:
    # model = load_model('./saved_models/lstm_aesop_1.h5')
    model = load_model('./saved_models/aesop_dropout_100.h5')

else:

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units, return_sequences = True)(x)
    x = LSTM(n_units)(x)
    x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)

    opti = RMSprop(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         416900    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         365568    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4169)              1071433   
Total params: 2,379,213
Trainable params: 2,379,213
Non-trainable params: 0
_________________________________________________________________


In [16]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "|":
            break
            
        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '
            
            
    return output_text

In [17]:
def on_epoch_end(epoch, logs):
    seed_text = ""
    gen_words = 500

    print('Temp 0.2')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
    print('Temp 0.33')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.33))
    print('Temp 0.5')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))
    print('Temp 1.0')
    print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))

    
    
if train_model:
    epochs = 1000
    batch_size = 32
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks = [callback], shuffle = True)




Epoch 1/1000
50396/50396 [==============================] - 145s 3ms/step - loss: 5.1582
Temp 0.2
the lion and the lion . a lion and a lion , and the a lion . 
Temp 0.33
the lion and the fox . a lion and a lion and a stag of a man . " 
Temp 0.5
the the the lion . a lion and a begged . a man , and the fox , who he his watched , and was his to . and he time , and , the wolf was a me , the ass , and had the and , " and was , and a body , " the one , to their frog , " 
Temp 1.0
the cage and and . a the ass her companions an gold . but they about have some way of strong , 
Epoch 2/1000
50396/50396 [==============================] - 135s 3ms/step - loss: 4.7787
Temp 0.2
the ass and the ass . a dog was the ass , and was the lion , and was the ass , and was the wolf , and he to the ass , and was the lion , and was the ass , and the ass , and the ass , and was the lion , and the ass , and to the ass , and was the lion , and was the ass , and the ass , and the fox , and the ass of the ass , and 

50396/50396 [==============================] - 129s 3ms/step - loss: 4.5448
Temp 0.2
the wolf and the fox . a man was a fox , and was a ass of a - own - own head , and was him to be a own tree of the ass of the ass . " the fox was be a ass , " he was a other came , and , " said the lion , " i have be a lion , and you have be a ass of the man of the own horse . " 
Temp 0.33
the lion and the fox . a fox was a loss was a ox , and was a dog , and , who was a flock , and was a own water of the than had the tree of the ass , and said , " i have be a lion of the fox , and , my own man , the lion was up to a became ass of the ass of than the lion of him . " 
Temp 0.5
the horse , the fox , and the ass . a fox , the ass was agreement to be to ; and the lion of her and be often a rest , and would himself to . " the wolf was all to a friend , " you cried , " i can be be , you , if a ass of the fox of the man , and should had be be , and you let them and one and grass it . " the edge was so said , 

the lion and the most ground . a cocks the man gaily a young mouse were wolf to free . but he did of them at me of he were and show them into a horse of his the . but they eagle the ass who lost very and plumage with their triumphantly , and the ass he came to long in a coming of the water of their a carried again . at they ass to be from just of the been sooner of ass : but , so he had one ; he only the friend he made out ox to if it was up of in an the came . but the lion saw to she of me was eat for the my hand , and this up how will enough a water to them ? when , it , one will ! they without with him in is . 
Epoch 11/1000
50396/50396 [==============================] - 132s 3ms/step - loss: 4.7195
Temp 0.2
the lion and the lion . a lion was the ass was a , and , " he was the , and , as he was the , and the lion was to the , and , in the , and , the , had , the , the lion was the , and he had and said , " i have a , i have to be the , you have be to the , for you , you , i have to 

50396/50396 [==============================] - 133s 3ms/step - loss: 4.8903
Temp 0.2
the lion and the lion . a dog was a , and the , was a , and the , was a , and said to him , " you have , : you have , you , be , your , your , is a , with your , ? you , is , to be , and , ! you , be , and , be , your , : be , and , be , ? is , is , and the , is to be . " 
Temp 0.33
the lion and the wolf . a lion , who was , and was the , and the lion was to the , in the , of the , of him , and was do to have the , 
Temp 0.5
the fox and the man . a ass was a , and a man who had been in a , had " them was a he had , and that that that the . 
Temp 1.0
the man and the dog . a fox was very in . " 
Epoch 15/1000
50396/50396 [==============================] - 133s 3ms/step - loss: 4.8345
Temp 0.2
the lion and the lion . a lion was the lion of a wolf in a lion of a lion of a lion of the lion . when the wolf was a wolf of his , when the lion was a lion of the wolf , and was a lion of a lion of a wolf . when th

50396/50396 [==============================] - 131s 3ms/step - loss: 5.0401
Temp 0.2
the lion and the fox . a ass was in a , when a fox was very the " so he had , when he , to the man of the , when the ass was the " but the ass was it at his the " so he said to him , " what a you , you are the " i am you , i have the " you have be you to be , for you , " 
Temp 0.33
the lion and the fox . a fox , who was , who had , who was , as he would , when the fox was it of the in the " and the fox had that he was , and the fox came up and was , and was to the " when he could be to the he i would " in the . " 
Temp 0.5
the lion and the fox . a ass was a great to the , when at last the ass was you , when a ass to be you , that he said to his i had , " but the house is the to you , and the ass is the the do . " 
Temp 1.0
the lion and the wolf . a wolf and a ass came into a destined when they was one day , who were always " you said to him , " if you will your what you have well if you are , : said th

...

the hare and the owl . an eagle and a certain man got down upon a hare ; but that he had no longer that it cried that before mouse enough , a very mouse in whenever i shall give yourself or the body . and , their teeth for them , for they let them so much thing , when they were doing as much as he could wish he would do his master to know nothing and returned to ; and the farmer came to look just caught his off for the longer of him while the ass into themselves and eaten of the hid . the farmer went to the hid about to the you together and always along the goats as had an goats . when it cried that she did from us , " as you could think what they would go to the lion for the so you ran upon the thief to his hands in the would . " at last of as he could in the the road . the fox to her , and with his same but told him his hands , he had got a little number of one for a few i man , and were was in a number of order to secure to last one could do so i must ; so he begged at her same of t

the farmer and the fox . a fox once fell into the ground , and out of its legs , which they were doing on to keep an ass . but one of his was him in great own time , and then he returned in a beautiful days as he went up in the river and came to look at it and said to him , " if you had been in the a beautiful and you are , is there , my friend , you see : for i am lion so that , what will have not not . 
Temp 1.0
the fox and the snake . a fox once fell to the ground and had a great deal for them , but his the i shall things by the land with all the river and a eye of the image . but before him , in last the ass took up and said to the ass , " we would replied , " don't show me my shall , i am doing at that you might can you get up in getting ! " he replied , " if i can get along in head a be you is , and i'll show you in your his life . " but the first replied , " don't show my dinner , i can't be sure ! and till they were : if you are bring will be better of be . " 
Epoch 195/1000
50

50396/50396 [==============================] - 139s 3ms/step - loss: 3.3537
Temp 0.2
the lion and the dog . a wolf once fell to the ground and had a sheep , which he begged he would be caught at an could his be i - i when i them to carry him again . but all the one were and make you again ? " the wolf took him in his den , but the way was soon away for that it was so fast a sheep to in the river . on the ground , being fast so his den , and took to be able to drink . he felt all the a beautiful good but , who all good as a man which had been in to all the rest , had 
Temp 0.33
the lion and the sheep . a wolf was once a lion and a lion were sitting on a moment , who caught himself and his fly down , when he would fly under the first and then in his most but the ass of a great deal : so he had " and as she could think , one of the wild goats that they was in , they thought the would seen the life - ass with them . so he went to his hands and asked him he as he was a hare of him . when th

the wolf , the snake , and the bat . a fowler and a lion took up in a number of the hare , which was so fast how can , ; but he could in a sheep . then the fox replied , " oh ! don't to then do that my husband , my friend of me ! i was only . a few i , sir , " said the sow that i will keep your tail , and you what , who are a large him . 
Epoch 201/1000
34080/50396 [===================>..........] - ETA: 42s - loss: 3.2936

KeyboardInterrupt: 

In [ ]:
model.summary()

In [126]:
seed_text = "the frog and the snake . "
gen_words = 500
temp = 0.1

print (generate_text(seed_text, gen_words, model, seq_length, temp))

the frog and the snake . a about a his way , who had a way to go - tree in a good going to give her a very much good . it is the time of the fox so much to the one of the better : and my in be the way was turned to work and they again . a lion and put them into the fox came and said , " you have just to be a better of together . you may do , the horse , and there not the ass to a very his very way from his neck . when he was after a the one he not to well again , and had i go with all it . " 


In [101]:
def generate_human_led_text(model, max_sequence_len):
    
    output_text = ''
    seed_text = start_story
    
    while 1:
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]

        top_10_idx = np.flip(np.argsort(probs)[-10:])
        top_10_probs = [probs[x] for x in top_10_idx]
        top_10_words = tokenizer.sequences_to_texts([[x] for x in top_10_idx])
        
        for prob, word in zip(top_10_probs, top_10_words):
            print('{:<6.1%} : {}'.format(prob, word))

        chosen_word = input()
                
        if chosen_word == '|':
            break
            
        
        seed_text += chosen_word + ' '
        output_text += chosen_word + ' '
        
        clear_output()

        print (output_text)
            
    
    

In [140]:
generate_human_led_text(model, 20)

the lion said happily , 
98.0%  : "
1.1%   : and
0.4%   : that
0.1%   : the
0.1%   : much
0.0%   : you
0.0%   : do
0.0%   : time
0.0%   : as
0.0%   : a


KeyboardInterrupt: 

In [20]:
# model.save('./saved_models/aesop_no_dropout_100.h5')